**TIP:** This training could take several hours. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```
Looks like this, it will click the screen every 10 minutes so that you don't get kicked off for being idle! HACKS!

In [1]:
import pandas as pd 
import re
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM ,SpatialDropout1D
from tensorflow.keras.layers import Bidirectional
from keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/AIM/clean_data.csv", lineterminator='\n')
df.head()

,id,txt,dialect
0,1175358310087892992,لكن بالنهاية ينتفض يغير,IQ
1,1175416117793349632,يعني هذا محسوب على البشر حيونه ووحشيه وتطلب...,IQ
2,1175450108898565888,مبين من كلامه خليجي,IQ
3,1175471073770573824,يسلملي مرورك وروحك الحلوه,IQ
4,1175496913145217024,وين هل الغيبه اخ محمد,IQ


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458000 entries, 0 to 457999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       458000 non-null  int64 
 1   txt      458000 non-null  object
 2   dialect  458000 non-null  object
dtypes: int64(1), object(2)
memory usage: 10.5+ MB


In [25]:
X = df['txt']
y = df['dialect']

In [26]:
np.unique(y)

array(['IQ', 'LY', 'QA', 'PL', 'SY', 'TN', 'JO', 'MA', 'SA', 'YE', 'DZ',
       'EG', 'LB', 'KW', 'OM', 'SD', 'AE', 'BH'], dtype=object)

In [24]:
encoder = LabelEncoder()

In [28]:
encoder.classes_ = np.load('/content/gdrive/MyDrive/Colab Notebooks/AIM/dl_classes.npy', allow_pickle=True)

In [29]:
y = encoder.transform(y)

In [31]:
np.unique(y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17])

In [32]:
voc_size=10000
sent_length=50
embedding_vector_features=50
def _txt_prep(X):
    features = X.to_list()

    onehot_repr=[one_hot(words,voc_size)for words in features] 

    embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
    return embedded_docs

embedded_docs = _txt_prep(X)

In [33]:
def __create_model():
  model = Sequential()
  model.add(Embedding(voc_size+1, embedding_vector_features, input_length=sent_length))
  model.add(SpatialDropout1D(0.2))
  model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
  model.add(Dense(18, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = __create_model()
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 50)            500050    
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 50, 50)           0         
 lDropout1D)                                                     
                                                                 
 bidirectional_2 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 18)                3618      
                                                                 
Total params: 624,468
Trainable params: 624,468
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
X_final=np.array(embedded_docs)
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.20, random_state=42)

In [36]:
checkpoint_path = "/content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint"
checkpoint_dir = os.path.dirname(checkpoint_path)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5)
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64,callbacks=[cp_callback,early_stopping])

Epoch 1/10
5725/5725 [==============================] - ETA: 0s - loss: 2.1874 - accuracy: 0.3100
Epoch 1: saving model to /content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint
5725/5725 [==============================] - 1534s 267ms/step - loss: 2.1874 - accuracy: 0.3100 - val_loss: 1.9866 - val_accuracy: 0.3771
Epoch 2/10
  16/5725 [..............................] - ETA: 23:47 - loss: 1.9575 - accuracy: 0.3838

KeyboardInterrupt: ignored

In [ ]:
# model.load_weights('/content/gdrive/MyDrive/Colab Notebooks/AIM/checkpoints/my_checkpoint')

In [37]:
model.evaluate(X_test,y_test)

2863/2863 [==============================] - 104s 36ms/step - loss: 1.9896 - accuracy: 0.3754


[1.9896355867385864, 0.3754148483276367]

In [38]:
y_pred=model.predict(X_test)

In [39]:
predict_classes=np.argmax(y_pred,axis=1)

In [41]:
predict_classes = encoder.inverse_transform(predict_classes)
y_test = encoder.inverse_transform(y_test)

In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predict_classes))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AE       0.27      0.15      0.19      5375
          BH       0.20      0.00      0.01      5243
          DZ       0.33      0.36      0.34      3285
          EG       0.55      0.77      0.64     11541
          IQ       0.48      0.36      0.41      3047
          JO       0.24      0.02      0.03      5680
          KW       0.27      0.60      0.37      8298
          LB       0.40      0.60      0.48      5500
          LY       0.39      0.56      0.46      7358
          MA       0.56      0.38      0.46      2333
          OM       0.32      0.07      0.11      3814
          PL       0.30      0.55      0.39      8644
          QA       0.38      0.30      0.34      6206
          SA       0.24      0.14      0.17      5403
          SD       0.52      0.41      0.46      2810
          SY       0.41      0.03      0.06      3248
          TN       0.52      0.09      0.15      1827
          YE       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
import os
import json
from datetime import datetime
from threading import Thread, current_thread, get_ident
from typing import Dict, List, Union
import joblib

report = classification_report(y_test, predict_classes, output_dict=True)
classes = encoder.classes_.tolist()
model_status = {}

def _update_status(status: str, classes: List[str] = [], evaluation: Dict = {}) -> None:
    model_status['status'] = status
    model_status['timestamp'] = datetime.now().isoformat()
    model_status['classes'] = classes
    model_status['evaluation'] = evaluation
    with open('dl_model_status.json', 'w+') as file:
        json.dump(model_status, file, indent=2)


_update_status("Model Ready", classes, report)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


['AE', 'BH', 'DZ', 'EG', 'IQ', 'JO', 'KW', 'LB', 'LY', 'MA', 'OM', 'PL', 'QA', 'SA', 'SD', 'SY', 'TN', 'YE']


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
